# Đánh giá các mô hình Embedding cho hệ thống Legal Chatbot

Notebook này đánh giá và so sánh hiệu suất của các mô hình embedding sau:
1. **thanhtantran/Vietnamese_Embedding_v2** - Mô hình chuyên cho tiếng Việt
2. **Baai/bge-m3** - Mô hình đa ngôn ngữ
3. **intfloat/multilingual-e5-large** - Mô hình đa ngôn ngữ lớn

Dataset: **anti-ai/ViNLI-Zalo-supervised** (triplet format: query, positive, hard_neg)

## 🚀 QUICK START cho Google Colab T4

**QUAN TRỌNG - Làm theo thứ tự:**

1. ✅ **Runtime → Restart runtime** (để clear memory cũ)
2. ✅ Chạy cell "Cài đặt thư viện"
3. ✅ Chạy cell "Import thư viện"
4. ✅ Chạy cell "Clear GPU Memory"
5. ✅ Load dataset
6. ✅ **Chọn cấu hình:**
   - Chạy cả 3 models: `MAX_SAMPLES=1000, BATCH_SIZE=4`
   - Chạy 1 model: `MAX_SAMPLES=2000, BATCH_SIZE=8`
7. ✅ Chạy các cell còn lại

**Nếu gặp OOM:** Restart runtime và giảm MAX_SAMPLES

## 1. Cài đặt thư viện

In [1]:
!pip install -q datasets transformers sentence-transformers torch scikit-learn numpy pandas tqdm matplotlib seaborn

## 2. Import thư viện

## ⚠️ GPU Memory Management (Quan trọng cho Colab T4!)

**HƯỚNG DẪN TRÁNH LỖI "CUDA OUT OF MEMORY" TRÊN COLAB T4 (15GB):**

### ✅ Bước 1: Restart Runtime trước khi chạy
- Menu: **Runtime → Restart runtime**
- Hoặc: **Ctrl+M .** (phím tắt)

### ✅ Bước 2: Clear GPU memory nếu cần
- Chạy cell "Clear GPU Memory" ở section 3

### ✅ Bước 3: Điều chỉnh cấu hình
**Trong cell config (section 3):**
```python
MAX_SAMPLES = 2000   # Giảm xuống 1000 nếu vẫn OOM
BATCH_SIZE = 8       # Giảm xuống 4 nếu vẫn OOM  
USE_FP16 = True      # BẮT BUỘC phải True
```

### ✅ Bước 4: Chạy từng model một
**Trong cell "Định nghĩa các mô hình" (section 4):**
- Uncomment một trong các dòng để chỉ chạy 1 model
- Ví dụ: `models_to_evaluate = {"BGE-M3": "BAAI/bge-m3"}`

### 📊 Khuyến nghị cấu hình cho Colab T4:
- **Chạy tất cả 3 models:** MAX_SAMPLES=1000, BATCH_SIZE=4, USE_FP16=True
- **Chạy 1 model:** MAX_SAMPLES=2000, BATCH_SIZE=8, USE_FP16=True
- **An toàn nhất:** MAX_SAMPLES=500, BATCH_SIZE=4, USE_FP16=True

In [ ]:
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import warnings
import gc
warnings.filterwarnings('ignore')

# Thiết lập device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Thiết lập memory management cho CUDA
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"Allocated GPU Memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"Cached GPU Memory: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

/home/mikeethanh/anaconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


## 3. Load Dataset

## 🔄 Clear GPU Memory (Chạy cell này nếu gặp OOM)

**Nếu bạn gặp lỗi "CUDA out of memory", chạy cell dưới đây để clear memory:**

In [ ]:
# Clear tất cả GPU memory
import torch
import gc

# Kill tất cả process đang dùng GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    
    # Force clear all allocated memory
    torch.cuda.synchronize()
    
    print("✅ GPU Memory cleared!")
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
    
# Nếu vẫn không được, restart runtime:
# Runtime -> Restart runtime (trong Colab menu)

In [3]:
# Load dataset từ HuggingFace
print("Loading dataset...")
dataset = load_dataset("anti-ai/ViNLI-Zalo-supervised")

# Kiểm tra cấu trúc dataset
print(f"\nDataset splits: {dataset.keys()}")
print(f"\nSample data:")
print(dataset['train'][0] if 'train' in dataset else dataset[list(dataset.keys())[0]][0])

Loading dataset...


Generating train split: 100%|██████████| 32980/32980 [00:00<00:00, 45453.00 examples/s]


Dataset splits: dict_keys(['train'])

Sample data:
{'query': 'Tổ sát_hạch cấp giấy_phép lái tàu_hỏa có bao_nhiêu thành_viên ?', 'positive': 'Điều 30 . Tổ sát_hạch 1 . Tổ sát_hạch do Cục_trưởng Cục Đường_sắt Việt_Nam thành_lập , chịu sự chỉ_đạo trực_tiếp của Hội_đồng sát_hạch . \n 2 . Tổ sát_hạch có ít_nhất 05 thành_viên , bao_gồm tổ_trưởng , các sát_hạch viên lý_thuyết và sát_hạch viên thực_hành . Tổ_trưởng Tổ sát_hạch là công_chức Cục Đường_sắt Việt_Nam hoặc lãnh_đạo doanh_nghiệp có thí_sinh dự kỳ sát_hạch , các sát_hạch viên là người đang công_tác tại doanh_nghiệp có thí_sinh tham_dự kỳ sát_hạch và người đang công_tác tại các cơ_sở đào_tạo liên_quan đến lái tàu . \n 3 . Tiêu_chuẩn của sát_hạch viên : \n a ) Có tư_cách đạo_đức tốt và có chuyên_môn phù_hợp ; \n b ) Đã qua khóa huấn_luyện về nghiệp_vụ sát_hạch lái tàu do Cục Đường_sắt Việt_Nam tổ_chức và được cấp thẻ sát_hạch viên ; \n c ) Sát_hạch viên lý_thuyết phải tốt_nghiệp đại_học trở lên chuyên_ngành phù_hợp nội_dung sát_hạch , 

In [ ]:
# ⚙️ CẤU HÌNH CHO GOOGLE COLAB T4 GPU (15GB)
MAX_SAMPLES = 2000  # Giảm xuống 2000 samples để tránh OOM trên T4
BATCH_SIZE = 8      # Giảm batch size xuống 8 để tiết kiệm memory
USE_FP16 = True     # Dùng FP16 (half precision) để giảm 50% memory

print("⚙️ CONFIGURATION FOR COLAB T4 GPU:")
print(f"  - Max samples: {MAX_SAMPLES}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Use FP16: {USE_FP16}")
print(f"  - Device: {device}")
print("\n💡 Nếu vẫn OOM, giảm MAX_SAMPLES xuống 1000 hoặc BATCH_SIZE xuống 4")

# Chuẩn bị dữ liệu - lấy split phù hợp
if 'test' in dataset:
    eval_data = dataset['test']
elif 'validation' in dataset:
    eval_data = dataset['validation']
elif 'train' in dataset:
    # Nếu chỉ có train, lấy một số mẫu để test
    eval_data = dataset['train'].select(range(min(MAX_SAMPLES, len(dataset['train']))))
else:
    # Lấy split đầu tiên
    split_name = list(dataset.keys())[0]
    eval_data = dataset[split_name].select(range(min(MAX_SAMPLES, len(dataset[split_name]))))

print(f"\n✅ Number of evaluation samples: {len(eval_data)}")
print(f"✅ Columns: {eval_data.column_names}")


Number of evaluation samples: 20000

Columns: ['query', 'positive', 'hard_neg']


## 4. Định nghĩa các mô hình cần đánh giá

In [ ]:
# Danh sách các mô hình cần đánh giá
models_to_evaluate = {
    "Vietnamese_Embedding_v2": "thanhtantran/Vietnamese_Embedding_v2",
    "BGE-M3": "BAAI/bge-m3",
    "Multilingual-E5-Large": "intfloat/multilingual-e5-large"
}

# 🔥 OPTION: Chỉ chạy 1 model để tránh OOM
# Uncomment dòng dưới để chỉ test 1 model
# models_to_evaluate = {"Vietnamese_Embedding_v2": "thanhtantran/Vietnamese_Embedding_v2"}
# models_to_evaluate = {"BGE-M3": "BAAI/bge-m3"}
# models_to_evaluate = {"Multilingual-E5-Large": "intfloat/multilingual-e5-large"}

print("Models to evaluate:")
for name, model_id in models_to_evaluate.items():
    print(f"  - {name}: {model_id}")
print(f"\n📊 Total: {len(models_to_evaluate)} model(s)")

Models to evaluate:
  - Vietnamese_Embedding_v2: thanhtantran/Vietnamese_Embedding_v2
  - BGE-M3: BAAI/bge-m3
  - Multilingual-E5-Large: intfloat/multilingual-e5-large


## 5. Hàm đánh giá

In [ ]:
def compute_embeddings(model: SentenceTransformer, texts: List[str], batch_size: int = 16) -> np.ndarray:
    """Tính embeddings cho một list các văn bản"""
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True  # Normalize để tính cosine similarity
    )
    return embeddings


def evaluate_triplet_ranking(queries_emb: np.ndarray, 
                             positives_emb: np.ndarray, 
                             negatives_emb: np.ndarray) -> Dict[str, float]:
    """Đánh giá mô hình dựa trên triplet ranking
    
    Args:
        queries_emb: Embeddings của queries
        positives_emb: Embeddings của positive documents
        negatives_emb: Embeddings của negative documents
    
    Returns:
        Dictionary chứa các metrics
    """
    # Tính similarity scores
    pos_scores = np.sum(queries_emb * positives_emb, axis=1)  # Cosine similarity (đã normalize)
    neg_scores = np.sum(queries_emb * negatives_emb, axis=1)
    
    # Metrics
    # 1. Accuracy: Tỷ lệ positive có score cao hơn negative
    accuracy = np.mean(pos_scores > neg_scores)
    
    # 2. Mean Positive Score
    mean_pos_score = np.mean(pos_scores)
    
    # 3. Mean Negative Score
    mean_neg_score = np.mean(neg_scores)
    
    # 4. Mean Score Difference (margin)
    mean_diff = np.mean(pos_scores - neg_scores)
    
    # 5. MRR (Mean Reciprocal Rank)
    # Trong trường hợp triplet, nếu positive rank 1 thì MRR = 1, rank 2 thì MRR = 0.5
    ranks = np.where(pos_scores > neg_scores, 1, 2)
    mrr = np.mean(1.0 / ranks)
    
    # 6. Recall@1: Tỷ lệ positive nằm ở top-1
    recall_at_1 = accuracy  # Giống accuracy trong trường hợp triplet
    
    return {
        "accuracy": accuracy,
        "mean_positive_score": mean_pos_score,
        "mean_negative_score": mean_neg_score,
        "mean_score_difference": mean_diff,
        "mrr": mrr,
        "recall@1": recall_at_1
    }


def evaluate_model(model_name: str, model_path: str, eval_data, batch_size: int = 16) -> Dict[str, float]:
    """Đánh giá một mô hình embedding
    
    Args:
        model_name: Tên mô hình
        model_path: Đường dẫn hoặc ID của mô hình trên HuggingFace
        eval_data: Dataset để đánh giá
        batch_size: Batch size cho encoding
    
    Returns:
        Dictionary chứa các metrics
    """
    print(f"\n{'='*60}")
    print(f"Evaluating: {model_name}")
    print(f"{'='*60}")
    
    # Clear memory trước khi load model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print(f"GPU Memory before loading: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    
    # Load model
    print(f"Loading model from {model_path}...")
    try:
        model = SentenceTransformer(model_path, device=device)
        
        # Dùng half precision (FP16) để giảm 50% memory trên T4
        if device == "cuda" and USE_FP16:
            model = model.half()
            print("✅ Using FP16 (half precision) to save memory")
        
        if torch.cuda.is_available():
            print(f"GPU Memory after loading: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        raise
    
    # Extract texts
    queries = [item['query'] for item in eval_data]
    positives = [item['positive'] for item in eval_data]
    negatives = [item['hard_neg'] for item in eval_data]
    
    # Compute embeddings
    print(f"\nComputing embeddings for {len(queries)} samples...")
    print("Encoding queries...")
    queries_emb = compute_embeddings(model, queries, batch_size)
    
    # Clear cache sau mỗi lần encode
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("Encoding positives...")
    positives_emb = compute_embeddings(model, positives, batch_size)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("Encoding negatives...")
    negatives_emb = compute_embeddings(model, negatives, batch_size)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Evaluate
    print("\nCalculating metrics...")
    metrics = evaluate_triplet_ranking(queries_emb, positives_emb, negatives_emb)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"  - Accuracy: {metrics['accuracy']:.4f}")
    print(f"  - MRR: {metrics['mrr']:.4f}")
    print(f"  - Recall@1: {metrics['recall@1']:.4f}")
    print(f"  - Mean Positive Score: {metrics['mean_positive_score']:.4f}")
    print(f"  - Mean Negative Score: {metrics['mean_negative_score']:.4f}")
    print(f"  - Mean Score Difference: {metrics['mean_score_difference']:.4f}")
    
    # Clear memory - QUAN TRỌNG!
    del model
    del queries_emb, positives_emb, negatives_emb
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    print(f"\n✅ Memory cleared after evaluating {model_name}")
    if torch.cuda.is_available():
        print(f"GPU Memory after cleanup: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    
    return metrics

## 6. Chạy đánh giá cho tất cả các mô hình

In [ ]:
# Lưu kết quả
results = {}

# Đánh giá từng mô hình
for model_name, model_path in models_to_evaluate.items():
    try:
        # Clear memory trước mỗi model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        metrics = evaluate_model(model_name, model_path, eval_data, batch_size=BATCH_SIZE)
        results[model_name] = metrics
        
        # Đợi một chút để GPU cleanup hoàn toàn
        import time
        time.sleep(2)
        
    except Exception as e:
        print(f"\n❌ Error evaluating {model_name}: {str(e)}")
        print(f"\n💡 Tip: Try reducing MAX_SAMPLES or BATCH_SIZE in the configuration cell")
        results[model_name] = None
        
        # Clear memory sau lỗi
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

print("\n" + "="*60)
print("Evaluation completed!")
print("="*60)

## 7. So sánh kết quả

In [ ]:
# Tạo DataFrame để so sánh
df_results = pd.DataFrame(results).T
df_results = df_results.sort_values('accuracy', ascending=False)

print("\n" + "="*80)
print("COMPARISON OF ALL MODELS")
print("="*80)
print(df_results.to_string())
print("\n")

# Tìm mô hình tốt nhất
best_model = df_results['accuracy'].idxmax()
print(f"\n🏆 BEST MODEL: {best_model}")
print(f"   Accuracy: {df_results.loc[best_model, 'accuracy']:.4f}")
print(f"   MRR: {df_results.loc[best_model, 'mrr']:.4f}")

## 8. Visualization

In [ ]:
# Thiết lập style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Tạo figure với subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Embedding Models Evaluation on ViNLI-Zalo Dataset', fontsize=16, fontweight='bold')

# 1. Accuracy comparison
ax1 = axes[0, 0]
df_results['accuracy'].plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_title('Accuracy Comparison', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=10)
ax1.set_xlabel('Model', fontsize=10)
ax1.set_ylim([0, 1])
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(df_results['accuracy']):
    ax1.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# 2. MRR comparison
ax2 = axes[0, 1]
df_results['mrr'].plot(kind='bar', ax=ax2, color='lightcoral', edgecolor='black')
ax2.set_title('Mean Reciprocal Rank (MRR)', fontsize=12, fontweight='bold')
ax2.set_ylabel('MRR', fontsize=10)
ax2.set_xlabel('Model', fontsize=10)
ax2.set_ylim([0, 1])
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate(df_results['mrr']):
    ax2.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# 3. Score comparison (Positive vs Negative)
ax3 = axes[1, 0]
x = np.arange(len(df_results))
width = 0.35
ax3.bar(x - width/2, df_results['mean_positive_score'], width, label='Positive', color='lightgreen', edgecolor='black')
ax3.bar(x + width/2, df_results['mean_negative_score'], width, label='Negative', color='salmon', edgecolor='black')
ax3.set_title('Mean Similarity Scores', fontsize=12, fontweight='bold')
ax3.set_ylabel('Cosine Similarity', fontsize=10)
ax3.set_xlabel('Model', fontsize=10)
ax3.set_xticks(x)
ax3.set_xticklabels(df_results.index, rotation=45, ha='right')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. Score Difference (Margin)
ax4 = axes[1, 1]
df_results['mean_score_difference'].plot(kind='bar', ax=ax4, color='plum', edgecolor='black')
ax4.set_title('Mean Score Difference (Margin)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Score Difference', fontsize=10)
ax4.set_xlabel('Model', fontsize=10)
ax4.grid(axis='y', alpha=0.3)
for i, v in enumerate(df_results['mean_score_difference']):
    ax4.text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('embedding_evaluation_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Visualization saved as 'embedding_evaluation_results.png'")

## 9. Chi tiết phân tích và khuyến nghị

In [ ]:
print("\n" + "="*80)
print("DETAILED ANALYSIS & RECOMMENDATION")
print("="*80)

# Ranking các mô hình theo từng metric
metrics_to_analyze = ['accuracy', 'mrr', 'mean_score_difference', 'mean_positive_score']

print("\n📊 Model Rankings by Metrics:")
for metric in metrics_to_analyze:
    sorted_models = df_results[metric].sort_values(ascending=False)
    print(f"\n{metric.upper().replace('_', ' ')}:")
    for i, (model, score) in enumerate(sorted_models.items(), 1):
        print(f"  {i}. {model}: {score:.4f}")

# Tính overall score (weighted average)
weights = {
    'accuracy': 0.4,
    'mrr': 0.3,
    'mean_score_difference': 0.2,
    'mean_positive_score': 0.1
}

df_results['overall_score'] = sum(
    df_results[metric] * weight 
    for metric, weight in weights.items()
)

best_overall = df_results['overall_score'].idxmax()

print(f"\n\n{'='*80}")
print(f"🏆 FINAL RECOMMENDATION: {best_overall}")
print(f"{'='*80}")
print(f"\nOverall Score: {df_results.loc[best_overall, 'overall_score']:.4f}")
print(f"\nKey Metrics:")
print(f"  • Accuracy: {df_results.loc[best_overall, 'accuracy']:.4f}")
print(f"  • MRR: {df_results.loc[best_overall, 'mrr']:.4f}")
print(f"  • Mean Score Difference: {df_results.loc[best_overall, 'mean_score_difference']:.4f}")
print(f"  • Mean Positive Score: {df_results.loc[best_overall, 'mean_positive_score']:.4f}")

print(f"\n💡 Recommendation for Vietnamese Legal Chatbot:")
print(f"   Use '{best_overall}' as the embedding model for your RAG system.")
print(f"\n   Model path: {models_to_evaluate[best_overall]}")

## 10. Lưu kết quả

In [ ]:
# Lưu kết quả ra CSV
output_file = 'embedding_evaluation_results.csv'
df_results.to_csv(output_file)
print(f"\n✅ Results saved to '{output_file}'")

# Lưu khuyến nghị
recommendation = f"""
EMBEDDING MODEL EVALUATION RESULTS
==================================
Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
Dataset: anti-ai/ViNLI-Zalo-supervised
Number of samples: {len(eval_data)}

RECOMMENDED MODEL: {best_overall}
Model Path: {models_to_evaluate[best_overall]}

Performance Metrics:
- Accuracy: {df_results.loc[best_overall, 'accuracy']:.4f}
- MRR: {df_results.loc[best_overall, 'mrr']:.4f}
- Mean Score Difference: {df_results.loc[best_overall, 'mean_score_difference']:.4f}
- Overall Score: {df_results.loc[best_overall, 'overall_score']:.4f}

ALL MODELS COMPARISON:
{df_results.to_string()}
"""

with open('embedding_recommendation.txt', 'w', encoding='utf-8') as f:
    f.write(recommendation)

print("✅ Recommendation saved to 'embedding_recommendation.txt'")
print("\n" + recommendation)